In [122]:
pip install jupyter openai minsearch requests

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [123]:
import requests 

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [124]:
documents[2]

{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
 'section': 'General course-related questions',
 'question': 'Course - Can I still join the course after the start date?',
 'course': 'data-engineering-zoomcamp'}

In [125]:
import minsearch

index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

index.fit(documents)

ImportError: cannot import name 'AppendableIndex' from 'minsearch' (/workspaces/RAG-Search-Engine/02-VectorSearch/minsearch.py)

In [4]:
from groq import Groq

openai_client = Groq(
    api_key=""
)

In [5]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )

    return results

In [6]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

<QUESTION>
{question}
</QUESTION>

<CONTEXT>
{context}
</CONTEXT>
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [7]:
def llm(prompt):
    response = openai_client.chat.completions.create(
        model='llama-3.3-70b-versatile',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [8]:
def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [9]:
rag('how do I run kafka?')

"To run Kafka, the instructions vary depending on the context. \n\nIf you're using Java, navigate to the project directory and run: \n```bash\njava -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java\n```\n\nIf you're using Python, first create a virtual environment and install the necessary packages by running:\n```bash\npython -m venv env\nsource env/bin/activate\npip install -r ../requirements.txt\n```\nThen, activate the virtual environment every time you need to run Kafka:\n```bash\nsource env/bin/activate\n```\nNote: For Windows, the path to activate the virtual environment is slightly different: `env/Scripts/activate`. \n\nAdditionally, ensure that the Docker images are up and running before attempting to run Kafka in the virtual environment. \n\nIt's also important to install the necessary dependencies, including `kafka-python-ng`, which can be installed using pip:\n```bash\npip install kafka-python-ng\n```"

In [10]:
rag('the course has already started, can I still enroll?')

"Yes, you can still enroll in the course even after it has started. You will still be eligible to submit homework, but be aware that there will be deadlines for turning in the final projects, so it's best not to leave everything for the last minute."

In [11]:
rag('are you happy?')

'There is no information in the provided context that indicates whether I, the course teaching assistant, am happy or not. The context only provides answers to specific questions related to the course, such as technical issues with dbt, dataset locations, and course-related questions, but does not address personal feelings or emotions.'

## RAG with Vector Search

In [12]:
from qdrant_client import QdrantClient, models

In [13]:
qd_client = QdrantClient("http://localhost:6333")

In [18]:
EMBEDDING_DIMENSIONALITY = 512
model_handle = "jinaai/jina-embeddings-v2-small-en"

In [19]:
collection_name = "zoomcamp-faq"

In [19]:
qd_client.delete_collection(collection_name=collection_name)

False

In [20]:
qd_client.create_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=EMBEDDING_DIMENSIONALITY,
        distance=models.Distance.COSINE
    )
)

True

In [21]:
qd_client.create_payload_index(
    collection_name=collection_name,
    field_name="course",
    field_schema="keyword"
)

UpdateResult(operation_id=1, status=<UpdateStatus.COMPLETED: 'completed'>)

In [22]:
points = []

for i, doc in enumerate(documents):
    text = doc['question'] + ' ' + doc['text']
    vector = models.Document(text=text, model=model_handle)
    point = models.PointStruct(
        id=i,
        vector=vector,
        payload=doc
    )
    points.append(point)

In [23]:
qd_client.upsert(
    collection_name=collection_name,
    points=points
)

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

config.json: 0.00B [00:00, ?B/s]

tokenizer_config.json:   0%|          | 0.00/367 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

onnx/model.onnx:   0%|          | 0.00/130M [00:00<?, ?B/s]

UpdateResult(operation_id=2, status=<UpdateStatus.COMPLETED: 'completed'>)

In [20]:
question = 'I just discovered the course. Can I still join it?'

In [58]:
def vector_search(question):
    print('vector_search is used')
    #course = ""
    query_points = qd_client.query_points(
        collection_name=collection_name,
        query=models.Document(
            text=question,
            model=model_handle 
        ),
        #query_filter=models.Filter( 
            #must=[
                #models.FieldCondition(
                    #key="course",
                    #match=models.MatchValue(value=course)
                #)
            #]
        #),
        limit=5,
        with_payload=True
    )
    
    results = []
    
    for point in query_points.points:
        results.append(point.payload)
    
    return results

In [59]:
def rag(query):
    search_results = vector_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [60]:
rag('how do I run kafka?')

vector_search is used


"To run Kafka, you need to start the Kafka broker docker container. If it's not working, use `docker ps` to confirm, then run `docker compose up -d` in the docker compose yaml file folder to start all instances. Additionally, make sure the `StreamsConfig.BOOTSTRAP_SERVERS_CONFIG` is set to the correct server URL and cluster key and secrets are updated in `Secrets.java`. \n\nFor running Kafka-related Java scripts (e.g., `JsonProducer.java`, `JsonConsumer.java`), use the command `java -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java` in the project directory.\n\nFor running Kafka-related Python scripts (e.g., `producer.py`), create a virtual environment, install packages with `pip install -r ../requirements.txt`, and run the script within the virtual environment. \n\nNote: The exact steps may vary depending on your operating system and specific setup."

In [61]:
rag('the course has already started, can I still enroll?')

vector_search is used


"Yes, you can still enroll in the course even after it has started. You won't be able to submit some of the homeworks, but you can still take part in the course. To be eligible for a certificate, you need to submit 2 out of 3 course projects and review 3 peers' projects by the deadline."

In [62]:
rag('are you happy?')

vector_search is used


'There is no information in the CONTEXT regarding the question of happiness. The CONTEXT only provides answers to technical questions related to machine learning, regression, and data processing, but does not address personal feelings or emotions such as happiness.'

# Agentic RAG

In [63]:
prompt_template = """
You're a course teaching assistant.

You're given a QUESTION from a course student and that you need to answer with your own knowledge and provided CONTEXT, use them templates below to answer.
At the beginning the context is EMPTY.

<QUESTION>
{question}
</QUESTION>

<CONTEXT> 
{context}
</CONTEXT>

If CONTEXT is EMPTY, you can use our FAQ database.
In this case, use the following output template as output:

{{
"action": "SEARCH",
"reasoning": "<add your reasoning here>"
}}

If you can answer the QUESTION using CONTEXT, use this template as output:

{{
"action": "ANSWER",
"answer": "<your answer>",
"source": "CONTEXT"
}}

If the context doesn't contain the answer, use your own knowledge to answer the question, use this template as output:

{{
"action": "ANSWER",
"answer": "<your answer>",
"source": "OWN_KNOWLEDGE"
}}
""".strip()

In [64]:
question = "how do I run docker on gentoo?"
context = "EMPTY"

prompt = prompt_template.format(question=question, context=context)
print(prompt)

You're a course teaching assistant.

You're given a QUESTION from a course student and that you need to answer with your own knowledge and provided CONTEXT, use them templates below to answer.
At the beginning the context is EMPTY.

<QUESTION>
how do I run docker on gentoo?
</QUESTION>

<CONTEXT> 
EMPTY
</CONTEXT>

If CONTEXT is EMPTY, you can use our FAQ database.
In this case, use the following output template as output:

{
"action": "SEARCH",
"reasoning": "<add your reasoning here>"
}

If you can answer the QUESTION using CONTEXT, use this template as output:

{
"action": "ANSWER",
"answer": "<your answer>",
"source": "CONTEXT"
}

If the context doesn't contain the answer, use your own knowledge to answer the question, use this template as output:

{
"action": "ANSWER",
"answer": "<your answer>",
"source": "OWN_KNOWLEDGE"
}


In [65]:
answer = llm(prompt)
print(answer)

{
"action": "ANSWER",
"answer": "To run Docker on Gentoo, you'll need to emerge the docker package and its dependencies. You can do this by running the command 'emerge -av docker' as the root user. After installation, you'll need to add your user to the docker group by running 'usermod -aG docker $USER' and then restart your system or log out and log back in. Once you've done this, you can start the Docker service by running 'rc-service docker start' and enable it to start at boot with 'rc-update add docker default'. You can then verify that Docker is running by running 'docker run -it --rm hello-world'. Make sure your kernel supports Docker by checking for the necessary kernel options, such as 'Containers' and 'Namespace support' under 'General setup' and 'Networking support' and 'Networking options' in your kernel configuration.",
"source": "OWN_KNOWLEDGE"
}


In [66]:
question = "how do I join the course?"
context = "EMPTY"

prompt = prompt_template.format(question=question, context=context)
answer_json = llm(prompt)
print(answer)

{
"action": "ANSWER",
"answer": "To run Docker on Gentoo, you'll need to emerge the docker package and its dependencies. You can do this by running the command 'emerge -av docker' as the root user. After installation, you'll need to add your user to the docker group by running 'usermod -aG docker $USER' and then restart your system or log out and log back in. Once you've done this, you can start the Docker service by running 'rc-service docker start' and enable it to start at boot with 'rc-update add docker default'. You can then verify that Docker is running by running 'docker run -it --rm hello-world'. Make sure your kernel supports Docker by checking for the necessary kernel options, such as 'Containers' and 'Namespace support' under 'General setup' and 'Networking support' and 'Networking options' in your kernel configuration.",
"source": "OWN_KNOWLEDGE"
}


In [67]:
import json

In [68]:
answer = json.loads(answer_json)
answer["action"]

'SEARCH'

In [69]:
def build_context(search_results):
    context = ""

    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

    return context.strip()

In [70]:
search_results = vector_search(question)
context = build_context(search_results)
prompt = prompt_template.format(question=question, context=context)
print(prompt)

vector_search is used
You're a course teaching assistant.

You're given a QUESTION from a course student and that you need to answer with your own knowledge and provided CONTEXT, use them templates below to answer.
At the beginning the context is EMPTY.

<QUESTION>
how do I join the course?
</QUESTION>

<CONTEXT> 
section: General course-related questions
question: The course has already started. Can I still join it?
answer: Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.
In order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.

section: General course-related questions
question: Adding community notes
answer: You can create your own github repository for the course with your notes, homework, projects, etc.
Then fork the o

In [71]:
answer = llm(prompt)
print(answer)

{
"action": "ANSWER",
"answer": "To join the course, you can go to the course page (http://mlzoomcamp.com/), scroll down and start going through the course materials. Then read everything in the cohort folder for your cohort’s year. You can also register before the course starts using the provided link, join the course Telegram channel with announcements, and register in DataTalks.Club's Slack and join the channel.",
"source": "CONTEXT"
}


In [72]:
def agentic_rag_v1(question):
    context = "EMPTY"
    prompt = prompt_template.format(question=question, context=context)
    answer_json = llm(prompt)
    answer = json.loads(answer_json)
    print(answer)

    if answer['action'] == 'SEARCH':
        print('need to perform search...')
        search_results = search(question)
        context = build_context(search_results)
        
        prompt = prompt_template.format(question=question, context=context)
        answer_json = llm(prompt)
        answer = json.loads(answer_json)
        print(answer)

    return answer

In [73]:
agentic_rag_v1('how do I join the course?')

{'action': 'SEARCH', 'reasoning': 'The context is empty, so I will search the FAQ database to find the steps to join the course.'}
need to perform search...
{'action': 'ANSWER', 'answer': "To join the course, you should register before the course starts using the provided link, subscribe to the course public Google Calendar, join the course Telegram channel with announcements, and register in DataTalks.Club's Slack and join the channel. The exact registration link is not provided in the context, but it's mentioned that you can still join and submit homework even after the course starts.", 'source': 'CONTEXT'}


{'action': 'ANSWER',
 'answer': "To join the course, you should register before the course starts using the provided link, subscribe to the course public Google Calendar, join the course Telegram channel with announcements, and register in DataTalks.Club's Slack and join the channel. The exact registration link is not provided in the context, but it's mentioned that you can still join and submit homework even after the course starts.",
 'source': 'CONTEXT'}

In [74]:
agentic_rag_v1('how patch KDE under FreeBSD?')

{'action': 'SEARCH', 'reasoning': 'The context is empty, and patching KDE under FreeBSD is a specific topic that requires detailed information. I will search our FAQ database to find the relevant instructions or guidelines for patching KDE under FreeBSD.'}
need to perform search...
{'action': 'ANSWER', 'answer': "To patch KDE under FreeBSD, you can use the ports collection or pkg command. First, ensure your ports tree is up to date by running 'portsnap fetch update' or 'git pull' if you're using git for ports. Then, you can update KDE by running 'pkg upgrade' or by compiling it from the ports tree using 'make install clean' in the appropriate KDE port directory. If you're using a custom or older version of KDE, you might need to specify the exact port or package name, such as 'kde5' or 'plasma5'. You can search for available KDE packages or ports using 'pkg search kde' or 'make search name=kde' in the ports tree. Always refer to the FreeBSD handbook or official KDE on FreeBSD forums fo

{'action': 'ANSWER',
 'answer': "To patch KDE under FreeBSD, you can use the ports collection or pkg command. First, ensure your ports tree is up to date by running 'portsnap fetch update' or 'git pull' if you're using git for ports. Then, you can update KDE by running 'pkg upgrade' or by compiling it from the ports tree using 'make install clean' in the appropriate KDE port directory. If you're using a custom or older version of KDE, you might need to specify the exact port or package name, such as 'kde5' or 'plasma5'. You can search for available KDE packages or ports using 'pkg search kde' or 'make search name=kde' in the ports tree. Always refer to the FreeBSD handbook or official KDE on FreeBSD forums for the most accurate and up-to-date instructions.",
 'source': 'OWN_KNOWLEDGE'}

## Agentic search

So far we had two actions only: search and answer.

But we can let our "agent" formulate one or more search queries - and do it for a few iterations until we found an answer

Let's build a prompt:

- List available actions:
    - Search in FAQ
    - Answer using own knowledge
    - Answer using information extracted from FAQ
- Provide access to the previous actions
- Have clear stop criteria (no more than X iterations)
- We also specify the output format, so it's easier to parse it

In [106]:
prompt_template = """
You're a course teaching assistant.

You're given a QUESTION from a course student and that you need to answer with your own knowledge and provided CONTEXT. Only use the output templates below.

The CONTEXT is build with the documents from our FAQ database.
SEARCH_QUERIES contains the queries that were used to retrieve the documents
from FAQ to and add them to the context.
PREVIOUS_ACTIONS contains the actions you already performed.

At the beginning the CONTEXT is empty.

You can perform the following actions:

- Search in the FAQ database to get more data for the CONTEXT
- Answer the question using the CONTEXT
- Answer the question using your own knowledge

For the SEARCH action, build search requests based on the CONTEXT and the QUESTION.
Carefully analyze the CONTEXT and generate the requests to deeply explore the topic. 

Don't use search queries used at the previous iterations.

Don't repeat previously performed actions.

Don't perform more than {max_iterations} iterations for a given student question.
The current iteration number: {iteration_number}. If we exceed the allowed number 
of iterations, give the best possible answer with the provided information.

IMPORTANT: Only respond with the JSON templates below. DO NOT add any explanation, intro, or extra text before or after the JSON. Output must ONLY be a valid JSON block.

If you want to perform search, only use this template as output:

{{
"action": "SEARCH",
"reasoning": "<add your reasoning here>",
"keywords": ["search query 1", "search query 2", ...]
}}

If you can answer the QUESTION using CONTEXT, only use this template as output:

{{
"action": "ANSWER_CONTEXT",
"answer": "<your answer>",
"source": "CONTEXT"
}}

If the context doesn't contain the answer, use your own knowledge to answer the question, only use this template as output:

{{
"action": "ANSWER",
"answer": "<your answer>",
"source": "OWN_KNOWLEDGE"
}}

<QUESTION>
{question}
</QUESTION>

<SEARCH_QUERIES>
{search_queries}
</SEARCH_QUERIES>

<CONTEXT> 
{context}
</CONTEXT>

<PREVIOUS_ACTIONS>
{previous_actions}
</PREVIOUS_ACTIONS>
""".strip()

In [149]:
question = "how do I join the course?"

search_queries = []
search_results = []
previous_actions = []
context = build_context(search_results)

prompt = prompt_template.format(
    question=question,
    context=context,
    search_queries="\n".join(search_queries),
    previous_actions='\n'.join([json.dumps(a) for a in previous_actions]),
    max_iterations=3,
    iteration_number=1
)
print(prompt)

You're a course teaching assistant.

You're given a QUESTION from a course student and that you need to answer with your own knowledge and provided CONTEXT. Only use the output templates below.

The CONTEXT is build with the documents from our FAQ database.
SEARCH_QUERIES contains the queries that were used to retrieve the documents
from FAQ to and add them to the context.
PREVIOUS_ACTIONS contains the actions you already performed.

At the beginning the CONTEXT is empty.

You can perform the following actions:

- Search in the FAQ database to get more data for the CONTEXT
- Answer the question using the CONTEXT
- Answer the question using your own knowledge

For the SEARCH action, build search requests based on the CONTEXT and the QUESTION.
Carefully analyze the CONTEXT and generate the requests to deeply explore the topic. 

Don't use search queries used at the previous iterations.

Don't repeat previously performed actions.

Don't perform more than 3 iterations for a given student q

In [150]:
answer_json = llm(prompt)
print(answer_json)

{
"action": "SEARCH",
"reasoning": "The question is about joining a course, so we need to search for information related to course enrollment or registration.",
"keywords": ["course registration", "enrollment process", "join a course"]
}


In [151]:
answer = json.loads(answer_json)
print(json.dumps(answer, indent=2))

{
  "action": "SEARCH",
  "reasoning": "The question is about joining a course, so we need to search for information related to course enrollment or registration.",
  "keywords": [
    "course registration",
    "enrollment process",
    "join a course"
  ]
}


In [152]:
search_queries = []
search_results = []
previous_actions = []

In [153]:
previous_actions.append(answer)

In [154]:
keywords = answer['keywords']
search_queries.extend(keywords)

for k in keywords:
    res = vector_search(k)
    search_results.extend(res)

vector_search is used
vector_search is used
vector_search is used


In [155]:
len(search_results)

15

In [156]:
def dedup_by_question(seq):
    seen = set()
    result = []
    for el in seq:
        key = el['question']
        if key in seen:
            continue
        seen.add(key)
        result.append(el)
    return result

search_results = dedup_by_question(search_results)

In [157]:
len(search_results)

10

Now let's make another iteration - use the same code as previously, but remove variable initialization and increase the iteration number:

In [158]:
# question = "how do I join the course?"

# search_queries = []
# search_results = []
# previous_actions = []

context = build_context(search_results)

prompt = prompt_template.format(
    question=question,
    context=context,
    search_queries="\n".join(search_queries),
    previous_actions='\n'.join([json.dumps(a) for a in previous_actions]),
    max_iterations=3,
    iteration_number=2
)
print(prompt)

You're a course teaching assistant.

You're given a QUESTION from a course student and that you need to answer with your own knowledge and provided CONTEXT. Only use the output templates below.

The CONTEXT is build with the documents from our FAQ database.
SEARCH_QUERIES contains the queries that were used to retrieve the documents
from FAQ to and add them to the context.
PREVIOUS_ACTIONS contains the actions you already performed.

At the beginning the CONTEXT is empty.

You can perform the following actions:

- Search in the FAQ database to get more data for the CONTEXT
- Answer the question using the CONTEXT
- Answer the question using your own knowledge

For the SEARCH action, build search requests based on the CONTEXT and the QUESTION.
Carefully analyze the CONTEXT and generate the requests to deeply explore the topic. 

Don't use search queries used at the previous iterations.

Don't repeat previously performed actions.

Don't perform more than 3 iterations for a given student q

In [159]:
answer_json = llm(prompt)
answer = json.loads(answer_json)
print(json.dumps(answer, indent=2))

{
  "action": "ANSWER_CONTEXT",
  "answer": "To join the course, you can register using the provided link, and then start learning and submitting homework without waiting for a confirmation email. You can also join the course Telegram channel with announcements and the course's Slack channel. The course materials are available on the course page, and you can start watching the videos and accessing the materials even before the course starts.",
  "source": "CONTEXT"
}


Let's put everything together:

In [161]:
question = "what do I need to do to be successful at module 1?"

search_queries = []
search_results = []
previous_actions = []


iteration = 0

while True:
    print(f'ITERATION #{iteration}...')

    context = build_context(search_results)
    prompt = prompt_template.format(
        question=question,
        context=context,
        search_queries="\n".join(search_queries),
        previous_actions='\n'.join([json.dumps(a) for a in previous_actions]),
        max_iterations=3,
        iteration_number=iteration
    )

    print(prompt)

    answer_json = llm(prompt)
    answer = json.loads(answer_json)
    print(json.dumps(answer, indent=2))

    previous_actions.append(answer)

    action = answer['action']
    if action != 'SEARCH':
        break

    keywords = answer['keywords']
    search_queries = list(set(search_queries) | set(keywords))
    
    for k in keywords:
        res = search(k)
        search_results.extend(res)

    search_results = dedup_by_question(search_results)
    
    iteration = iteration + 1
    if iteration >= 4:
        break

    print()

ITERATION #0...
You're a course teaching assistant.

You're given a QUESTION from a course student and that you need to answer with your own knowledge and provided CONTEXT. Only use the output templates below.

The CONTEXT is build with the documents from our FAQ database.
SEARCH_QUERIES contains the queries that were used to retrieve the documents
from FAQ to and add them to the context.
PREVIOUS_ACTIONS contains the actions you already performed.

At the beginning the CONTEXT is empty.

You can perform the following actions:

- Search in the FAQ database to get more data for the CONTEXT
- Answer the question using the CONTEXT
- Answer the question using your own knowledge

For the SEARCH action, build search requests based on the CONTEXT and the QUESTION.
Carefully analyze the CONTEXT and generate the requests to deeply explore the topic. 

Don't use search queries used at the previous iterations.

Don't repeat previously performed actions.

Don't perform more than 3 iterations for a

In [165]:
def agentic_search(question):
    search_queries = []
    search_results = []
    previous_actions = []

    iteration = 0
    
    while True:
        print(f'ITERATION #{iteration}...')
    
        context = build_context(search_results)
        prompt = prompt_template.format(
            question=question,
            context=context,
            search_queries="\n".join(search_queries),
            previous_actions='\n'.join([json.dumps(a) for a in previous_actions]),
            max_iterations=3,
            iteration_number=iteration
        )
    
        # print(prompt)
    
        answer_json = llm(prompt)
        answer = json.loads(answer_json)
        print(json.dumps(answer, indent=2))

        previous_actions.append(answer)
    
        action = answer['action']
        if action != 'SEARCH':
            break
    
        keywords = answer['keywords']
        search_queries = list(set(search_queries) | set(keywords))

        for k in keywords:
            res = search(k)
            search_results.extend(res)
    
        search_results = dedup_by_question(search_results)
        
        iteration = iteration + 1
        if iteration >= 4:
            break
    
        print()

    return answer

In [166]:
agentic_search('how do I prepare for the course?')

ITERATION #0...
{
  "action": "SEARCH",
  "reasoning": "The student is asking about course preparation, so I need to gather information on what steps they can take to get ready for the course. I'll search for general course preparation tips and materials.",
  "keywords": [
    "course preparation tips",
    "study materials",
    "academic readiness"
  ]
}

ITERATION #1...
{
  "action": "ANSWER",
  "answer": "To prepare for the course, you can start by installing and setting up all the dependencies and requirements such as Google cloud account, Google Cloud SDK, Python 3, Terraform, and Git. You can also look over the prerequisites and syllabus to see if you are comfortable with these subjects. Additionally, you can join the course Telegram channel with announcements and register in DataTalks.Club's Slack and join the channel. It's also recommended to subscribe to the course public Google Calendar to stay updated on the course schedule.",
  "source": "OWN_KNOWLEDGE"
}


{'action': 'ANSWER',
 'answer': "To prepare for the course, you can start by installing and setting up all the dependencies and requirements such as Google cloud account, Google Cloud SDK, Python 3, Terraform, and Git. You can also look over the prerequisites and syllabus to see if you are comfortable with these subjects. Additionally, you can join the course Telegram channel with announcements and register in DataTalks.Club's Slack and join the channel. It's also recommended to subscribe to the course public Google Calendar to stay updated on the course schedule.",
 'source': 'OWN_KNOWLEDGE'}

## Function Calling (Tool Use)

In [202]:
def vector_search(query: str):
    print('vector_search is used')
    #course = ""
    query_points = qd_client.query_points(
        collection_name=collection_name,
        query=models.Document(
            text=question,
            model=model_handle 
        ),
        #query_filter=models.Filter( 
            #must=[
                #models.FieldCondition(
                    #key="course",
                    #match=models.MatchValue(value=course)
                #)
            #]
        #),
        limit=5,
        with_payload=True
    )
    
    results = []
    
    for point in query_points.points:
        results.append(point.payload)
    
    return results

In [203]:
search_tool = {
    "type": "function",
    "function": {
        "name": "vector_search",
        "description": "Search the FAQ database",
        "parameters": {
            "type": "object",
            "properties": {
                "query": {
                    "type": "string",
                    "description": "Search query text to look up in the course FAQ."
                }
            },
            "required": ["query"]
        }
    }
}


In [204]:
client = Groq(
    api_key=""
)

In [245]:
question = "How can I join this course?"

developer_prompt = """
You're a course teaching assistant. 
You're given a question from a course student and your task is to answer it.
""".strip()

tools = [search_tool]

messages = [
    {"role": "system", "content": developer_prompt},
    {"role": "user", "content": question}
]

response = client.chat.completions.create(
    model='llama3-70b-8192',
    messages=messages,
    tools=tools,
    tool_choice="auto",
    max_completion_tokens=4096, 
    temperature=0.5
)
response.choices[0]

Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', executed_tools=None, function_call=None, reasoning=None, tool_calls=[ChatCompletionMessageToolCall(id='xc4kjcx64', function=Function(arguments='{"query":"enrollment"}', name='vector_search'), type='function')]))

In [209]:
response_message = response.choices[0].message
tool_calls = response_message.tool_calls
tool_calls

[ChatCompletionMessageToolCall(id='yqwxesxhy', function=Function(arguments='{"query":"How to do well in module 1"}', name='vector_search'), type='function')]

In [247]:
# Process tool calls
messages.append(response_message)

available_functions = {
    "vector_search": vector_search
}

for tool_call in tool_calls:
    function_name = tool_call.function.name
    function_to_call = available_functions[function_name]
    function_args = json.loads(tool_call.function.arguments)
    function_response = function_to_call(**function_args)

    messages.append(
        {
            "role": "tool",
            "content": str(function_response),
            "tool_call_id": tool_call.id,
        }
    )

# print(messages)

# Make the final request with tool call results
final_response = client.chat.completions.create(
    model='llama3-70b-8192',
    messages=messages,
    tools=tools,
    tool_choice="auto",
    max_completion_tokens=4096
)

print(final_response.choices[0].message.content)

vector_search is used
Based on your question "How can I join this course?", the most relevant answer from the tool call result is:

'Welcome to the course! Go to the course page (http://mlzoomcamp.com/), scroll down and start going through the course materials. Then read everything in the cohort folder for your cohort’s year.\nClick on the links and start watching the videos. Also watch office hours from previous cohorts. Go to DTC youtube channel and click on Playlists and search for {course yyyy}. ML Zoomcamp was first launched in 2021.\nOr you can just use this link: http://mlzoomcamp.com/#syllabus'

Follow these steps to join the course!
